## Import Dependencies

In [1]:
#import liberaries
import import_ipynb
import time
from datetime import timedelta, datetime
from pubnub.callbacks import SubscribeCallback
from pubnub.enums import PNStatusCategory, PNOperationType

#import ipynb files
import common.Cassandra_DB_Setup as Cassandra_DB_Setup

## Initialize Global Variables

In [ ]:
# initialize global variables
get_hour = None
get_date = None

set_minutes = [1, 5, 10]
set_top_trends = None
set_zip_path = None
set_cassandra_configs = None

creation_time = None
start_times = [None, None, None]
end_time = None
session = None

tweet_count = dict()
trends_tweets = dict()

## Function to Publish Callback

In [ ]:
def my_publish_callback(envelope, status):
    
    # Check whether request successfully completed or not
    
    if not status.is_error():
        
        pass  # Message successfully published to specified channel.
    
    else:
        
        pass  # Handle message publish error. Check 'category' property to find out possible issue
        # because of which request did fail.
        # Request can be resent using: [status retry];

## Initialize Class for Subscibing Callback for Retreiving Start Time

In [ ]:
class FirstSubscribeCallback(SubscribeCallback):
    
    def presence(self, pubnub, presence):
        
        pass  # handle incoming presence data
    

    def status(self, pubnub, status):
        
        if status.category == PNStatusCategory.PNUnexpectedDisconnectCategory:
            
            pass  # This event happens when radio / connectivity is lost

        elif status.category == PNStatusCategory.PNConnectedCategory:
            
            # Connect event. You can do stuff like publish, and know you'll get it.
            # Or just use the connected event to confirm you are subscribed for
            # UI / internal notifications, etc
            
            pubnub.publish().channel('pubnub-twitter').message("Hello").pn_async(my_publish_callback)

        elif status.category == PNStatusCategory.PNReconnectedCategory:
            
            pass
        
            # Happens as part of our regular operation. This event happens when
            # radio / connectivity is lost, then regained.
            
        elif status.category == PNStatusCategory.PNDecryptionErrorCategory:
            
            pass
        
            # Handle message decryption error. Probably client configured to
            # encrypt messages and on live data feed it received plain text.
    
    def message(self, pubnub, message):
        
        # Handle new message stored in message.message
        
        
        # Get Start Time
        
        creation_time = datetime.strptime(message.message['created_at'], "%a %b %d %H:%M:%S %z %Y")
        

        # Declare Global Variables
        
        global get_hour, get_date
        
        
        # Update global variables based on stream date and hour
        
        if get_hour is None :
            get_hour = creation_time.hour
            get_date = creation_time.date()
            print(get_hour, get_date)
            return

## Initialize Class for Subscibing Callback for Retreiving Tweets Counts for Top Trends

In [ ]:
class MainSubscribeCallback(SubscribeCallback):
    
    def status(self, pubnub, status):
        
         # handle according to status
            
        FirstSubscribeCallback.status(self, pubnub, status)
        
    def presence(self, pubnub, status):
        
         # handle incoming presence data
            
        FirstSubscribeCallback.presence(self, pubnub, presence)
        
    def message(self, pubnub, message):
        
        # Handle new message stored in message.message
        
        
        # Declare Global Variables
        
        global creation_time, end_time, start_times, tweet_count, set_top_trends, zip_path, cassandra_configs, trends_tweets, session
        
        
        # Get Tweet DateTime
        
        creation_time = datetime.strptime(message.message['created_at'], "%a %b %d %H:%M:%S %z %Y")
                
    
        # Start Processing Start Time are initialized
        
        if start_times[0] is not None:
            
            
            # Loop for the number of minutes to track data for
            
            for i in range(len(set_minutes)):
                
                
                # if new minutes are added, populate start time for them
                
                if start_times[i] is None:
                    start_times[i] = creation_time
                    
                    
                # Start Processing if set minutes have passed 
                
                if creation_time == (start_times[i]  + timedelta(minutes=set_minutes[i])) :
                    print(set_minutes[i], " minutes have passed ", creation_time)

                    
                    # Get unix time of current time and generate key (unix_time _ minutes)
                    
                    unix_time = int(time.mktime(start_times[i].timetuple())) 
                    key =  str(unix_time) + "_" + str(set_minutes[i])

                    
                    # Populate Payload ({Top Trend: Tweet Count})
                    
                    value = dict()
                    for trendd, tweet_numbers in tweet_count.items():
                        value[trendd] = tweet_numbers
                        trends_tweets[key] = value

                    print("Here's twitter tweets", trends_tweets)
                    
                    
                    # Insert the Key Value Pair in Casssandra DB table
                    
                    columns = '(StreamTimeKey, TweetCountForTrends)'
                    Cassandra_DB_Setup.insert_into_table_from_df(session, "twitter_trends", "tweets_per_trend", columns, trends_tweets)

                    
                    # Resrt Start time and Key Value Pair
                    
                    start_times[i] = creation_time
                    trends_tweets = dict()
            
        else:
            
            
            # Initialize Start Times for the First Time
            
            for i in range(len(set_minutes)):
                start_times[i] = creation_time
                
                
            # Connect to Cassandra DB    
            
            session = Cassandra_DB_Setup.cassandra_connection(set_zip_path, set_cassandra_configs)
            print("Process Started at: ", creation_time)
            print(session)
            
            
            # Initialze Key Value Pairs
            
            for trend_number, set_trend in set_top_trends.items():
                tweet_count[set_trend] = 0
                
                
                
        # Get hashtags from stream   
        
        trend_list = message.message['entities']['hashtags']
        
        
        # Start Processing If hastags exist
        
        if trend_list:
            
            
            # Loop For each Hastag
            
            for trend in trend_list:

                
                # Get hasgtag text and convert to lower case
                
                trend_text = trend['text'].lower()
                
                
                # Loop for each Top Trend
                
                for trend_number, set_trend in set_top_trends.items():
                    
                    # If Top Trend found in current Hashtag text 
                    if set_trend == trend_text:
                        
                        
                        # Increae Tweet Count for Top Trend
                        
                        print(set_trend, trend_text)
                        tweet_count[set_trend] +=1